source code from https://www.tensorflow.org/tutorials/keras/text_classification_with_hub

## Imports

In [1]:
import io
import os
import re
import shutil
import string
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow.keras as keras
import tensorflow_datasets as tfds

from datetime import datetime
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Activation, Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [2]:
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.3.0
Eager mode:  True
Hub version:  0.9.0
GPU is NOT AVAILABLE


## Gathering data

In [3]:
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteJILN13/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteJILN13/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteJILN13/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


<body>Visualizing some data</body>

<body>loading the embedding model</body>

In [4]:
hub_layer = hub.KerasLayer("https://tfhub.dev/google/Wiki-words-500-with-normalization/2",
                           input_shape=[], dtype=tf.string)

model = keras.Sequential()
model.add(hub_layer)
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 500)               504687500 
_________________________________________________________________
dense (Dense)                (None, 16)                8016      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 504,695,533
Trainable params: 8,033
Non-trainable params: 504,687,500
_________________________________________________________________


<body>Defining the loss function</body>

In [5]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

## Model

In [6]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=10,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/10
30/30 [==============================] - 4s 119ms/step - loss: 0.6926 - accuracy: 0.5007 - val_loss: 0.6855 - val_accuracy: 0.4992
Epoch 2/10
30/30 [==============================] - 3s 111ms/step - loss: 0.6767 - accuracy: 0.5564 - val_loss: 0.6664 - val_accuracy: 0.5933
Epoch 3/10
30/30 [==============================] - 3s 113ms/step - loss: 0.6615 - accuracy: 0.6271 - val_loss: 0.6549 - val_accuracy: 0.6456
Epoch 4/10
30/30 [==============================] - 3s 113ms/step - loss: 0.6531 - accuracy: 0.6525 - val_loss: 0.6492 - val_accuracy: 0.6777
Epoch 5/10
30/30 [==============================] - 3s 113ms/step - loss: 0.6474 - accuracy: 0.6735 - val_loss: 0.6441 - val_accuracy: 0.6835
Epoch 6/10
30/30 [==============================] - 3s 112ms/step - loss: 0.6426 - accuracy: 0.6896 - val_loss: 0.6396 - val_accuracy: 0.6864
Epoch 7/10
30/30 [==============================] - 3s 112ms/step - loss: 0.6379 - accuracy: 0.7011 - val_loss: 0.6354 - val_accuracy: 0.7095
Epoch 

In [7]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
    print("%s: %.3f" % (name, value))

49/49 - 3s - loss: 0.6260 - accuracy: 0.7399
loss: 0.626
accuracy: 0.740
